In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import opendatasets as od

In [14]:
od.download("https://www.kaggle.com/datasets/zsinghrahulk/covertype-forest-cover-types")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

Skipping, found downloaded files in "./covertype-forest-cover-types" (use force=True to force download)
cuda


In [15]:
path = r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/gradient_decent/covertype-forest-cover-types/covertype.csv"
data_df = pd.read_csv(path)
data_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [16]:
data_df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Elevation', 'Aspect',
       'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type1',
       'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6',
       'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
       'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15',
       'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19',
       'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23',
       'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27',
       'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31',
       'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35',
       'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
       'Soil_Type40', 'Cover_Type'],
 

In [17]:
data_df = data_df.drop(columns=(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2']), axis=1)

In [18]:
data_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,258,0,510,221,232,148,6279,1,0,0,...,0,0,0,0,0,0,0,0,0,5
1,212,-6,390,220,235,151,6225,1,0,0,...,0,0,0,0,0,0,0,0,0,5
2,268,65,3180,234,238,135,6121,1,0,0,...,0,0,0,0,0,0,0,0,0,2
3,242,118,3090,238,238,122,6211,1,0,0,...,0,0,0,0,0,0,0,0,0,2
4,153,-1,391,220,234,150,6172,1,0,0,...,0,0,0,0,0,0,0,0,0,5


In [19]:
columns=data_df.columns
print(list(columns))

['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']


In [20]:
numerical_cols = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am']
binary_cols = ['Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']
label=['Cover_Type']



In [21]:
print("\n Checking the NAN values")
print(data_df.isnull().sum())

print("\n Checking the empty strings and unusual values")
print(data_df.isin(["", "NA", "N/A", "NONE", "none", "na", "NULL", "null", "None"]).sum())

print("\n Checking the duplicates")
print(data_df.duplicated().sum())


 Checking the NAN values
Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area                       0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0
Soil_Type12                           0
Soil_Type13                           0
Soil_Type14   

In [22]:
data_df.dropna(inplace=True)

In [23]:
scaler = StandardScaler()
data_df[numerical_cols] = scaler.fit_transform(data_df[numerical_cols])
X = data_df.iloc[:,:-1]
y=data_df[label]

In [29]:
data_df.shape

(581012, 52)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)


In [31]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [32]:
X_train.shape, X_test.shape, X_val.shape

((522910, 51), (29051, 51), (29051, 51))

In [42]:
class dataset(Dataset):
    def __init__(self,X ,y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [43]:
training_dataset = dataset(X_train, y_train)
validation_dataset = dataset(X_val, y_val)
testing_dataset = dataset(X_test, y_test)

In [44]:
train_dataloader = DataLoader(training_dataset, batch_size=64, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=64, shuffle=False)
testing_dataloader = DataLoader(testing_dataset, batch_size= 64, shuffle= False)

In [47]:
for x , y in train_dataloader:
    print(x)
    print(y)
    break

tensor([[ 0.3414,  0.5932, -0.9890,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6280,  2.6517, -0.7671,  ...,  0.0000,  0.0000,  0.0000],
        [-1.2676, -0.7963, -0.5606,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-1.2676, -0.7963, -0.5119,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1485, -0.2988,  1.3140,  ...,  0.0000,  1.0000,  0.0000],
        [-1.2676, -0.7963,  0.4713,  ...,  0.0000,  0.0000,  0.0000]])
tensor([[3.],
        [1.],
        [2.],
        [2.],
        [3.],
        [1.],
        [2.],
        [2.],
        [6.],
        [2.],
        [2.],
        [1.],
        [7.],
        [1.],
        [1.],
        [1.],
        [1.],
        [6.],
        [1.],
        [7.],
        [6.],
        [1.],
        [2.],
        [2.],
        [2.],
        [2.],
        [1.],
        [2.],
        [2.],
        [2.],
        [1.],
        [1.],
        [3.],
        [5.],
        [2.],
        [1.],
        [2.],
        [2.],
        [3.],
        [2.],
      